### Use ``Open Route Service API`` to retrieve distance matrix
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
from add_distance import add_distance_time

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(70444, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2013]
print(place_df.shape)
place_df.head(10)

(5929, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
0,TAHARA BRIDGE RESERVE PICNIC AREA,-37.66568,141.66459,2013,park,217011420
1,CULGOA RECREATION RESERVE,-35.71682,143.10101,2013,park,215031400
2,CUMBERLAND,-38.57263,143.94681,2013,park,203031049
3,WAREENA PARK,-36.35911,146.30977,2013,park,204021066
4,ENTERPRIZE PARK,-37.82000,144.95973,2013,park,206041505
5,RYANS RESERVE,-37.66807,145.15741,2013,park,209031215
6,FERNTREE GULLY PICNIC GROUND,-37.89088,145.31692,2013,park,211051282
7,LORRAINE KOVACS RESERVE,-37.97763,145.25403,2013,park,212021454
8,RAY BASTIN RESERVE,-38.03359,145.30791,2013,park,212021456
9,NEVILLE HAMILTON RESERVE,-38.02612,145.34533,2013,park,212021293


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'price', 'type', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2/2013_property_with_SA2.csv')[PROPERTY_COLS]
# 2013 example
print(property_df.shape)
property_df.head(10)

(11466, 13)


,address,latitude,longitude,nbed,nbath,ncar,price,type,postcode,year,month,residence_type,SA2_CODE
0,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,$300_pw,House,3451,2013,11,House,202021027
1,"171 MAIN ROAD, CHEWTON",-37.081326,144.259245,3.0,1.0,8,$330_pw,House,3451,2013,9,House,202021027
2,"173 MAIN ROAD, CHEWTON",-37.080952,144.259378,2.0,1.0,0,$270_pw,House,3451,2013,10,House,202021027
3,"173 MAIN ROAD, CHEWTON",-37.080952,144.259378,2.0,1.0,0,$270_pw,House,3451,2013,9,House,202021027
4,"4 GEORGE STREET, CHEWTON",-37.082086,144.256513,3.0,1.0,2,$350_pw,House,3451,2013,4,House,202021027
5,"2/56 ALBERT ROAD, CHILTERN",-36.149080,146.604709,2.0,1.0,1,$185_pw,House,3683,2013,10,House,204031070
6,"8 SKERRY STREET, CHILTERN",-36.151158,146.598197,3.0,2.0,2,$280_Weekly,House,3683,2013,7,House,204031070
7,"4 GLENDALE DRIVE, CHIRNSIDE PARK",-37.751383,145.312411,4.0,2.0,2,$350_per_week,Rental_residential,3116,2013,2,Other,211051275
8,"12-14 CAMPUS GATE, CHIRNSIDE PARK",-37.755523,145.332948,5.0,2.0,2,$390_pw,House,3116,2013,8,House,211051275
9,"12-14 CAMPUS GATE, CHIRNSIDE PARK",-37.755523,145.332948,5.0,2.0,2,$390_per_week,House,3116,2013,6,House,211051275


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(merged_df.shape)
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

(153905, 18)


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,price,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
6010,"95 LINDENOW SOUTH FERNBANK ROAD, LINDEONW SOUT...",-37.834039,147.421859,1.0,3.0,1,$285,none,3875,2013,10,Other,205021082,MOSSIFACE PRIMARY SCHOOL,-37.72679,147.80709,primary
6011,"95 LINDENOW SOUTH FERNBANK ROAD, LINDEONW SOUT...",-37.834039,147.421859,1.0,3.0,1,$285,none,3875,2013,10,Other,205021082,NICHOLSON PRIMARY SCHOOL,-37.81578,147.74791,primary
6012,"95 LINDENOW SOUTH FERNBANK ROAD, LINDEONW SOUT...",-37.834039,147.421859,1.0,3.0,1,$285,none,3875,2013,10,Other,205021082,SARSFIELD PRIMARY SCHOOL,-37.76346,147.70514,primary
6013,"95 LINDENOW SOUTH FERNBANK ROAD, LINDEONW SOUT...",-37.834039,147.421859,1.0,3.0,1,$285,none,3875,2013,10,Other,205021082,TAMBO CROSSING PRIMARY SCHOOL,-37.50818,147.83376,primary
6014,"95 LINDENOW SOUTH FERNBANK ROAD, LINDEONW SOUT...",-37.834039,147.421859,1.0,3.0,1,$285,none,3875,2013,10,Other,205021082,TAMBO UPPER PRIMARY SCHOOL,-37.76620,147.84520,primary
6015,"95 LINDENOW SOUTH FERNBANK ROAD, LINDEONW SOUT...",-37.834039,147.421859,1.0,3.0,1,$285,none,3875,2013,10,Other,205021082,OMEO HIGHER ELEMENTARY SCHOOL,-37.09707,147.59764,secondary
6016,"215 SNOBBS LANE, LINDENOW SOUTH",-37.812368,147.404758,4.0,2.0,3,$250_per_week,House,3875,2013,8,House,205021082,LINDENOW SOUTH GOLF COURSE,-37.83235,147.43070,park
6017,"215 SNOBBS LANE, LINDENOW SOUTH",-37.812368,147.404758,4.0,2.0,3,$250_per_week,House,3875,2013,8,House,205021082,FAIRY DELL SCENIC RESERVE,-37.68929,147.77709,park
6018,"215 SNOBBS LANE, LINDENOW SOUTH",-37.812368,147.404758,4.0,2.0,3,$250_per_week,House,3875,2013,8,House,205021082,FERNBANK RECREATION RESERVE,-37.87370,147.32268,park
6019,"215 SNOBBS LANE, LINDENOW SOUTH",-37.812368,147.404758,4.0,2.0,3,$250_per_week,House,3875,2013,8,House,205021082,DEN OF NARGUN PICNIC GROUND,-37.70235,147.36070,park


#### Specify the ``year`` here

In [6]:
added_distance_merged_df = add_distance_time(merged_df, 2013) # specify the year here

Executing SA2 Code 202021027
locations coordinates = 
[(144.261809, -37.080111), (144.259245, -37.081326), (144.259378, -37.080952), (144.259378, -37.080952), (144.256513, -37.082086), (144.204949, -37.102357), (144.203967, -37.080944), (144.203822, -37.09104), (144.204893, -37.083365), (144.206394, -37.064804), (144.227191, -37.059802), (144.220049, -37.063354), (144.238524, -37.071009), (144.220335, -37.063373), (144.218941, -37.054819), (144.216771, -37.057411), (144.2256552, -37.0776772), (144.213082, -37.057264), (144.1971861, -37.069087), (144.24035, -37.073748), (144.24035, -37.073748), (144.201259982313, -37.0817900019732), (144.21378998508, -37.0631599986733), (144.258479992296, -37.0848499997293), (144.207489980437, -37.0701600003064), (144.218989994741, -37.0578299989792), (144.222329978283, -37.0666000002522), (144.261159985099, -37.0823299998888), (144.204659996604, -37.0929300012648), (144.22750002004, -37.0505400004888), (144.220979981831, -37.0634599991444), (144.962379

ValueError: Length of values (153038) does not match length of index (153905)

In [ ]:
added_distance_merged_df.head(20)